In [ ]:
import requests # pyhton package that makes web requests easy

In [ ]:
# there are various types of requests that we can make. One of them is ".get() request"
response = requests.get('https://google.com')

In [ ]:
# 200 : good OK
# 300 : redirected
# 400 : problem with the request
# 500 : problems with the server
print(response.status_code)
print(response.reason)

In [ ]:
response1 = requests.get('https://python.org/about/app')
print(response1.status_code)
print(response1.reason)

In [ ]:
response.content # we get the strings of bytes, This is HTML

In [ ]:
# URL is in two parts:
# Base URL : 'https://www.google.com/search'
# query string : (The part after the '?')
url = 'https://www.google.com/search?q=python&source=hp&ei=jdMfYYGeHYq9mAWLpbeQAQ&iflsig=AINFCbYAAAAAYR_hneWvEItyErGfWAt_29sS1T_PC8kG&oq=pyhto&gs_lcp=Cgdnd3Mtd2l6EAMYADIHCAAQsQMQCjIHCAAQsQMQCjIHCAAQsQMQCjIHCAAQsQMQCjIHCAAQsQMQCjIHCAAQsQMQCjIHCAAQsQMQCjIHCAAQsQMQCjIHCC4QsQMQCjIHCAAQsQMQCjoOCAAQ6gIQjwEQjAMQ5QI6DgguEOoCEI8BEIwDEOUCOgsIABCABBCxAxCDAToICAAQgAQQsQM6BQguEIAEOgUIABCABDoICC4QgAQQsQM6BwgAEIAEEApQ5RxYpiRgly9oAXAAeACAAYoBiAGPBZIBAzAuNZgBAKABAbABCg&sclient=gws-wiz'

In [ ]:
response = requests.get(url)

In [ ]:
response.content

In [ ]:
url_base = 'https://dog-facts-api.herokuapp.com/api/v1/resources/dogs'
response = requests.get(url_base + '?number=2')

In [ ]:
response.status_code

In [ ]:
response.reason

In [ ]:
response.content

In [ ]:
# This is a JSON string and not a python dictionary.
# We can decode this JSON string into python object like dictionary
response.text

In [ ]:
import json

# Loads that JSON string to be a Pyhton object
data = json.loads(response.text)

In [ ]:
data

In [ ]:
print(type(data))
print(type(data[0]))

In [ ]:
data[0]['fact']

In [ ]:
# Shorthand method:
# If the response is in the form of the JSON, we can convert it to a Pyhton object using .json()
# No need to do import json
data = response.json()
data

In [ ]:
url_base = 'https://dog-facts-api.herokuapp.com/api/v1/resources/dogs' # before '?'
params = {'number' : 3} # after '?' in the form of dictionary
response = requests.get(url_base, params=params)

In [ ]:
response.url

In [ ]:
response.text

In [ ]:
response.content

In [ ]:
response.status_code

In [ ]:
response.reason

In [ ]:
response.json()

## requests reponse based upon ip address

In [ ]:
response = requests.get('https://api.ipify.org', params={'format' : 'json'})
response.status_code

In [ ]:
response.text

In [ ]:
print(response.text)
# Inside the print statement, response.text takes care of the formatting

In [ ]:
response.content

In [ ]:
print(response.content)

In [ ]:
response.json()

In [ ]:
response.json()['ip']

In [ ]:
# Finding the location based on the IP address
base_url = 'https://api.techniknews.net/ipgeo/'
loc_response = requests.get(base_url + response.json()['ip'])
# ip_addr = response.josn()['ip']
# loc_response = requests.get(f'https://api.techniknews.net/ipgeo/{ip_addr}')
loc_response.json()

# Web scrapping

first we obtain the HTML using requests.get(url of web page)



In [ ]:
response = requests.get('https://webscraper.io/test-sites/e-commerce/static/computers/laptops')
response.status_code

In [ ]:
response.reason

In [ ]:
response.text

In [ ]:
# There is apackage called BeautifulSoup
from bs4 import BeautifulSoup

soup = BeautifulSoup(response.text)
soup

In [ ]:
# Parsing soup
soup.select('div.caption') # find all div tags with class value 'caption'

In [ ]:
el_products = soup.find_all('div', class_='caption')

In [ ]:
type(el_products)

In [ ]:
el_products[0]

In [ ]:
type(el_products[0])

In [ ]:
el_products[0].select('h4')

In [ ]:
el_products[0].select('h4')[0]

In [ ]:
price = el_products[0].select_one('h4.price')
price

In [ ]:
type(price)

In [ ]:
price.text

In [ ]:
type(price.text)

In [ ]:
price.text.replace('$', '')

In [ ]:
# Lets pull out the title
title_el = el_products[0].select_one('a')
title_el

In [ ]:
title_el.text # content

In [ ]:
title_el['title'] # value of title of the title attribute

In [ ]:
# Pulling out description
el_products[0].select_one('p').text

In [ ]:
# Lets use the dictionary to store the data/information we scrapped
# Lets use functions to encapsulate our work

def pull_info(product_el):
    """Return a dctionary of title, description, and price contained in HTML element."""
    title = product_el.select_one('a').text
    description = product_el.select_one('p').text
    price = product_el.select_one('h4.price').text.replace('$', '')
    
    return {
        'title' : title,
        'price' : price,
        'description' : description
    }

In [ ]:
pull_info(el_products[4])

In [ ]:
[pull_info(el) for el in el_products]

In [ ]:
# Lets encapsulate the whole process of getting list, right from getting the URL

def get_response(url, page):
    params = {'page' : page}
    return requests.get(url, params=params)

def create_soup(url, page):
    response = get_response(url, page)
    return BeautifulSoup(response.text)

def create_list(url, page):
    soup = create_soup(url, page)
    
    el_products = soup.select('div.caption')
    prod_list = []
        
    return [{
        'title': el.select_one('a').text,
        'description' : el.select_one('p').text,
        'price' : el.select_one('h4.price').text.replace('$', '')
    } for el in el_products ]

def scrape_page(url, page):
    prods_details = create_list(url, page)
    return prods_details

def scrape_all(base_url, page_limit=5):
    products = [prod for page in range(1, page_limit+1) for prod in scrape_page(base_url, page)]
    print(f'Total Products are {len(products)} in number.')
    return products
    
def dump_data(data):
    with open('output.json', 'w') as f:
        json.dump(data, f, indent=2)

In [ ]:
url = 'https://webscraper.io/test-sites/e-commerce/static/computers/laptops'
page_limit = 20
data = scrape_all(url, page_limit)

In [ ]:
dump_data(data)